<a href="https://colab.research.google.com/github/mad-scientist-in-training/BIOF509-Final-Project/blob/Project-Code/BIOF509_Final_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is a project to determine if speech patterns in Parkinson's patients vs healthy controls can be used to train a Support Vector Machine model to predict which patients belong to which category.

In [ ]:
#Code outline

In [38]:
#to download the kaggle dataset, create a kaggle folder under google drive, create a new API token under account in kaggle, put the kaggle.json file into the kaggle folder in your drive, and mount the drive
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLEC_CONFIG_DIR'] = "content/gdrive/My Drive/kaggle_dataset"
%cd /content/gdrive/My Drive/kaggle_dataset
#copy the API command from kaggle and unzip the file
!kaggle datasets download -d ruslankl/early-biomarkers-of-parkinsons-disease
!unzip early-biomarkers-of-parkinsons-disease

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/kaggle_dataset
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
Archive:  early-biomarkers-of-parkinsons-disease.zip
replace dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset.xls? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
#manipulate/visualize dataset to keep features you are interested in and get rid of ones you don't care about
import pandas as pd
PD_df = pd.read_csv(r'dataset.csv')
print(PD_df)
len(PD_df)
PD_df.head()

In [ ]:
#drop the first 40 columns, as they contain info we don't need
PD_df_1 = PD_df.drop(PD_df.columns[1:41], axis=1)
print(PD_df_1)



In [98]:
#Participant code needs to be made into distinct classes: P (Parkinson's disease) vs H (Healthy Control), which will be the target
import numpy as np
PD_DF = PD_df_1.rename({' Participant  code ':'Participant_Type'}, axis=1)
print(PD_DF.columns)
PD_DF['class'] = PD_DF.Participant_Type.str[0]
print(PD_DF)
target = PD_DF.pop('class')
PD_DF.insert(0, 'class', target)
PD_DF_new = PD_DF.drop(PD_DF.columns[1], axis=1)
PD_DF_new

Index(['Participant_Type', ' Entropy  of  speech  timing  (-) ',
       ' Rate  of  speech  timing  (-/min) ',
       ' Acceleration  of  speech  timing  (-/min2) ',
       ' Duration  of  pause  intervals  (ms) ',
       ' Duration  of  voiced  intervals  (ms) ',
       ' Gaping  in-between  voiced  intervals  (-/min) ',
       ' Duration  of  unvoiced  stops  (ms) ',
       ' Decay  of  unvoiced  fricatives  (‰/min) ',
       ' Relative  loudness  of  respiration  (dB) ',
       ' Pause  intervals  per  respiration  (-) ',
       ' Rate  of  speech  respiration  (-/min) ',
       ' Latency  of  respiratory  exchange  (ms) ',
       ' Entropy  of  speech  timing  (-) .1',
       ' Rate  of  speech  timing  (-/min) .1',
       ' Acceleration  of  speech  timing  (-/min2) .1',
       ' Duration  of  pause  intervals  (ms) .1',
       ' Duration  of  voiced  intervals  (ms) .1',
       ' Gaping  in-between  voiced  Intervals  (-/min) ',
       ' Duration  of  unvoiced  stops  (ms) .1',
 

,class,Entropy of speech timing (-),Rate of speech timing (-/min),Acceleration of speech timing (-/min2),Duration of pause intervals (ms),Duration of voiced intervals (ms),Gaping in-between voiced intervals (-/min),Duration of unvoiced stops (ms),Decay of unvoiced fricatives (‰/min),Relative loudness of respiration (dB),Pause intervals per respiration (-),Rate of speech respiration (-/min),Latency of respiratory exchange (ms),Entropy of speech timing (-) .1,Rate of speech timing (-/min) .1,Acceleration of speech timing (-/min2) .1,Duration of pause intervals (ms) .1,Duration of voiced intervals (ms) .1,Gaping in-between voiced Intervals (-/min),Duration of unvoiced stops (ms) .1,Decay of unvoiced fricatives (‰/min) .1,Relative loudness of respiration (dB) .1,Pause intervals per respiration (-) .1,Rate of speech respiration (-/min) .1,Latency of respiratory exchange (ms) .1
0,P,1.564,354,6.05,146,264,58.65,31.38,-2.101,-22.47,4.5,21.14,167,1.564,333,-2.82,158,318,49.01,22.37,0.588,-19.77,6.0,13.81,127
1,P,1.564,340,27.52,173,253,48.26,22.38,-1.745,-24.59,7.0,15.28,163,1.569,285,8.20,295,264,40.56,26.88,-0.825,-23.26,4.0,21.77,313
2,P,1.550,211,11.97,377,322,47.54,38.12,2.657,-16.89,3.0,20.76,372,1.550,247,4.71,280,317,48.97,22.37,-0.955,-13.29,4.0,22.52,201
3,P,1.519,140,-2.49,360,663,13.72,44.88,-0.934,-25.54,1.0,18.71,119,1.539,112,-9.09,397,800,18.69,49.37,0.791,-25.08,2.0,14.37,151
4,P,1.543,269,6.72,211,328,42.90,47.12,-0.973,-22.61,5.0,16.26,78,1.560,230,11.77,206,480,33.54,26.87,0.075,-22.32,5.0,14.61,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,H,1.564,265,3.58,198,365,40.25,26.88,-1.872,-28.04,6.5,10.24,158,1.554,254,8.43,264,380,36.36,26.88,0.023,-23.12,3.0,15.41,177
126,H,1.547,291,6.31,183,359,39.59,31.37,-1.517,-22.87,5.0,13.46,224,1.569,300,4.09,177,359,47.06,22.37,-0.117,-18.67,5.0,16.94,138
127,H,1.540,298,-13.66,177,283,53.01,50.50,-1.111,-22.91,4.5,19.11,251,1.537,246,5.09,223,391,37.74,40.37,0.411,-15.56,3.0,19.51,156
128,H,1.560,359,-2.44,169,256,50.68,17.88,-0.823,-23.82,6.5,18.14,226,1.561,262,2.80,252,361,38.92,26.88,0.389,-17.53,4.0,17.51,154


In [99]:
import numpy as np
PD_DF_new["class"] = PD_DF_new["class"].replace('P', 0)
PD_DF_new["class"] = PD_DF_new["class"].replace('H', 1)
PD_DF_new["class"] = PD_DF_new["class"].replace('R', 2)
PD_DF_new

,class,Entropy of speech timing (-),Rate of speech timing (-/min),Acceleration of speech timing (-/min2),Duration of pause intervals (ms),Duration of voiced intervals (ms),Gaping in-between voiced intervals (-/min),Duration of unvoiced stops (ms),Decay of unvoiced fricatives (‰/min),Relative loudness of respiration (dB),Pause intervals per respiration (-),Rate of speech respiration (-/min),Latency of respiratory exchange (ms),Entropy of speech timing (-) .1,Rate of speech timing (-/min) .1,Acceleration of speech timing (-/min2) .1,Duration of pause intervals (ms) .1,Duration of voiced intervals (ms) .1,Gaping in-between voiced Intervals (-/min),Duration of unvoiced stops (ms) .1,Decay of unvoiced fricatives (‰/min) .1,Relative loudness of respiration (dB) .1,Pause intervals per respiration (-) .1,Rate of speech respiration (-/min) .1,Latency of respiratory exchange (ms) .1
0,0,1.564,354,6.05,146,264,58.65,31.38,-2.101,-22.47,4.5,21.14,167,1.564,333,-2.82,158,318,49.01,22.37,0.588,-19.77,6.0,13.81,127
1,0,1.564,340,27.52,173,253,48.26,22.38,-1.745,-24.59,7.0,15.28,163,1.569,285,8.20,295,264,40.56,26.88,-0.825,-23.26,4.0,21.77,313
2,0,1.550,211,11.97,377,322,47.54,38.12,2.657,-16.89,3.0,20.76,372,1.550,247,4.71,280,317,48.97,22.37,-0.955,-13.29,4.0,22.52,201
3,0,1.519,140,-2.49,360,663,13.72,44.88,-0.934,-25.54,1.0,18.71,119,1.539,112,-9.09,397,800,18.69,49.37,0.791,-25.08,2.0,14.37,151
4,0,1.543,269,6.72,211,328,42.90,47.12,-0.973,-22.61,5.0,16.26,78,1.560,230,11.77,206,480,33.54,26.87,0.075,-22.32,5.0,14.61,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,1,1.564,265,3.58,198,365,40.25,26.88,-1.872,-28.04,6.5,10.24,158,1.554,254,8.43,264,380,36.36,26.88,0.023,-23.12,3.0,15.41,177
126,1,1.547,291,6.31,183,359,39.59,31.37,-1.517,-22.87,5.0,13.46,224,1.569,300,4.09,177,359,47.06,22.37,-0.117,-18.67,5.0,16.94,138
127,1,1.540,298,-13.66,177,283,53.01,50.50,-1.111,-22.91,4.5,19.11,251,1.537,246,5.09,223,391,37.74,40.37,0.411,-15.56,3.0,19.51,156
128,1,1.560,359,-2.44,169,256,50.68,17.88,-0.823,-23.82,6.5,18.14,226,1.561,262,2.80,252,361,38.92,26.88,0.389,-17.53,4.0,17.51,154


In [100]:
col_names = list(PD_DF_new)
col_names

['class',
 ' Entropy  of  speech  timing  (-) ',
 ' Rate  of  speech  timing  (-/min) ',
 ' Acceleration  of  speech  timing  (-/min2) ',
 ' Duration  of  pause  intervals  (ms) ',
 ' Duration  of  voiced  intervals  (ms) ',
 ' Gaping  in-between  voiced  intervals  (-/min) ',
 ' Duration  of  unvoiced  stops  (ms) ',
 ' Decay  of  unvoiced  fricatives  (‰/min) ',
 ' Relative  loudness  of  respiration  (dB) ',
 ' Pause  intervals  per  respiration  (-) ',
 ' Rate  of  speech  respiration  (-/min) ',
 ' Latency  of  respiratory  exchange  (ms) ',
 ' Entropy  of  speech  timing  (-) .1',
 ' Rate  of  speech  timing  (-/min) .1',
 ' Acceleration  of  speech  timing  (-/min2) .1',
 ' Duration  of  pause  intervals  (ms) .1',
 ' Duration  of  voiced  intervals  (ms) .1',
 ' Gaping  in-between  voiced  Intervals  (-/min) ',
 ' Duration  of  unvoiced  stops  (ms) .1',
 ' Decay  of  unvoiced  fricatives  (‰/min) .1',
 ' Relative  loudness  of  respiration  (dB) .1',
 ' Pause  intervals  per  

In [101]:
PD_DF_new['class']

0      0
1      0
2      0
3      0
4      0
      ..
125    1
126    1
127    1
128    1
129    1
Name: class, Length: 130, dtype: int64

In [118]:
#scale the data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
col_names = list(PD_DF_new)
df_scaled = PD_DF_new.copy()
all_cols_no_class = df_scaled[col_names]
all_cols_no_class = PD_DF_new.loc[:, PD_DF_new.columns !='class']
scaler = StandardScaler().fit(all_cols_no_class.values)
scaled_all_cols_no_class = scaler.transform(all_cols_no_class.values)
df_scaled = pd.DataFrame(scaled_all_cols_no_class)
df_scaled = pd.concat([df_scaled, PD_DF_new['class']], axis=1)
df_scaled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,class
0,0.651886,0.566139,-0.403129,-0.445840,-0.423163,0.346530,0.089869,-0.178231,-0.064241,-0.444838,0.596968,0.512796,0.625076,0.847663,-0.881266,-0.895194,-0.046626,0.314284,-0.727826,0.918717,0.080062,0.864238,-0.897053,-0.214423,0
1,0.651886,0.269543,1.370556,0.137207,-0.637062,-0.321809,-0.966776,-0.050858,-0.709701,0.593383,-0.550862,0.449428,1.000931,-0.063363,1.328958,0.830470,-0.788832,-0.363137,-0.135680,-1.476150,-0.982046,-0.256470,1.066252,1.976929,0
2,-0.059404,-2.463372,0.085936,4.542448,0.704673,-0.368123,0.881178,1.524128,1.634656,-1.067770,0.522536,3.760425,-0.427318,-0.784592,0.628987,0.641529,-0.060370,0.311077,-0.727826,-1.696485,2.052114,-0.256470,1.251237,0.657405,0
3,-1.634405,-3.967534,-1.108637,4.175345,7.335567,-2.543600,1.674836,0.239308,-0.998940,-1.898346,0.120991,-0.247624,-1.254200,-3.346853,-2.138808,2.115271,6.578256,-2.116413,2.817176,1.262778,-1.535925,-1.377178,-0.758931,0.068332,0
4,-0.415050,-1.234619,-0.347779,0.957791,0.821345,-0.666592,1.937823,0.225355,-0.106866,-0.237193,-0.358904,-0.897150,0.324392,-1.107247,2.044975,-0.290582,2.179994,-0.925917,-0.136993,0.049243,-0.695977,0.303884,-0.699736,0.068332,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0.651886,-1.319361,-0.607181,0.677065,1.540827,-0.837053,-0.438453,-0.096297,-1.760094,0.385739,-1.538075,0.370218,-0.126634,-0.651734,1.375088,0.439991,0.805537,-0.699843,-0.135680,-0.038891,-0.939440,-0.816824,-0.502419,0.374650,1
126,-0.211824,-0.768541,-0.381650,0.353150,1.424154,-0.879508,0.088695,0.030718,-0.186026,-0.237193,-0.907356,1.415796,1.000931,0.221333,0.504637,-0.655869,0.516901,0.157956,-0.727826,-0.276174,0.414824,0.303884,-0.125050,-0.084826,1
127,-0.567469,-0.620243,-2.031416,0.223584,-0.053699,-0.016264,2.334652,0.175980,-0.198205,-0.444838,0.199341,1.843532,-1.404542,-0.803572,0.705201,-0.076449,0.956728,-0.589211,1.635509,0.618723,1.361287,-0.816824,0.508832,0.127240,1
128,0.448660,0.672066,-1.104507,0.050830,-0.578726,-0.166142,-1.495098,0.279023,-0.475265,0.385739,0.009342,1.447480,0.399563,-0.499896,0.245908,0.288838,0.544391,-0.494613,-0.135680,0.581436,0.761759,-0.256470,0.015539,0.103677,1


In [130]:
#clean up the data so that there's no class 2
df_scaled_new = df_scaled.drop(df_scaled.loc[df_scaled['class']==2].index)
df_new_1 = df_scaled_new.rename(columns={'class':'24'})
print(df_new_1)

            0         1         2         3  ...        21        22        23  24
0    0.651886  0.566139 -0.403129 -0.445840  ...  0.864238 -0.897053 -0.214423   0
1    0.651886  0.269543  1.370556  0.137207  ... -0.256470  1.066252  1.976929   0
2   -0.059404 -2.463372  0.085936  4.542448  ... -0.256470  1.251237  0.657405   0
3   -1.634405 -3.967534 -1.108637  4.175345  ... -1.377178 -0.758931  0.068332   0
4   -0.415050 -1.234619 -0.347779  0.957791  ...  0.303884 -0.699736  0.068332   0
..        ...       ...       ...       ...  ...       ...       ...       ...  ..
125  0.651886 -1.319361 -0.607181  0.677065  ... -0.816824 -0.502419  0.374650   1
126 -0.211824 -0.768541 -0.381650  0.353150  ...  0.303884 -0.125050 -0.084826   1
127 -0.567469 -0.620243 -2.031416  0.223584  ... -0.816824  0.508832  0.127240   1
128  0.448660  0.672066 -1.104507  0.050830  ... -0.256470  0.015539  0.103677   1
129  0.042208 -1.340546 -0.366779  0.094018  ... -0.256470 -0.840324 -0.179078   1

[80

In [136]:
df_new_1.loc[:,'24']

0      0
1      0
2      0
3      0
4      0
      ..
125    1
126    1
127    1
128    1
129    1
Name: 24, Length: 80, dtype: int64

In [138]:
#split the data into training and test sets
import sklearn
from sklearn.model_selection import train_test_split
X = df_new_1.iloc[:, 0:23].values
y = df_new_1.iloc[:, 24].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [141]:
#train the algorithm
from sklearn.ensemble import RandomForestClassifier
regressor = RandomForestClassifier(n_estimators = 20, random_state=42)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)


In [143]:
#calculate score for how well the model fits
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_pred, y_test))

0.625
[[2 5]
 [1 8]]
0.625
